In [2]:
!pip install --upgrade transformers datasets accelerate peft bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 42.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [5]:
model_name = "google/gemma-2b"
dataset_name = "imdb"
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                                       # The rank of the LoRA matrices. Lower is smaller, faster, but less expressive.
    lora_alpha=16,                             # The scaling factor for the LoRA matrices. A common practice is 2*r.
    lora_dropout=0.1,                          # Dropout for the LoRA layers
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none"                                # We are not training any bias parameters
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Tokenizer pad token set to EOS token.")

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
# Load the IMDB dataset
dataset = load_dataset(dataset_name)

def formatting_func(example):
    sentiment = 'positive' if example['label'] == 1 else 'negative'
    prompt_template = f"Review: {{text}}\nSentiment: {sentiment}{tokenizer.eos_token}"

    max_review_tokens = 512 - len(tokenizer("\nSentiment: " + sentiment + tokenizer.eos_token)['input_ids']) - 10
    inputs = tokenizer(example['text'], truncation=True, max_length=max_review_tokens)
    truncated_text = tokenizer.decode(inputs['input_ids'], skip_special_tokens=True)

    return prompt_template.format(text=truncated_text)

small_train_dataset = dataset["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
base_model.config.pad_token_id = tokenizer.pad_token_id

print("\n--- Model has been loaded. Now checking GPU memory usage. ---")
!nvidia-smi

lora_model = get_peft_model(base_model, lora_config)

lora_model.print_trainable_parameters()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

training_args = TrainingArguments(
    output_dir="./lora_imdb_classifier_results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=lora_model,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    peft_config=lora_config,
    formatting_func=formatting_func,
    args=training_args,
)

trainer.train()

lora_model.to("cuda")
lora_model.eval()

# Test with a positive review
positive_review = "This movie was absolutely incredible! The performances were brilliant and the story was captivating."
prompt = f"Review: {positive_review}\nSentiment:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = lora_model.generate(**inputs, max_new_tokens=10, do_sample=False)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
predicted_sentiment = generated_text.split("Sentiment:")[1].strip().split()[0]

print(f"\nReview: '{positive_review}'")
print(f"Predicted Sentiment: {predicted_sentiment}")

# Test with a negative review
negative_review = "I was really let down by this film. The plot was predictable and the acting was wooden."
prompt = f"Review: {negative_review}\nSentiment:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = lora_model.generate(**inputs, max_new_tokens=10, do_sample=False)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
predicted_sentiment = generated_text.split("Sentiment:")[1].strip().split()[0]

print(f"\nReview: '{negative_review}'")
print(f"Predicted Sentiment: {predicted_sentiment}")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,2.881500,2.858362,2.834415,132373.000000,0.431460
2,2.832400,2.848953,2.799427,264746.000000,0.431342
3,2.753700,2.847910,2.801224,397119.000000,0.431081



Review: 'This movie was absolutely incredible! The performances were brilliant and the story was captivating.'
Predicted Sentiment: positive

Review: 'I was really let down by this film. The plot was predictable and the acting was wooden.'
Predicted Sentiment: negative
